<a href="https://colab.research.google.com/github/jonathanluo23/NBA-allstar-all-nba-Prediction/blob/main/XGB_Hypertuning_Allstar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Capstone/features_allstar')
data

,PTS,FTA,FTM,PIE,FGM,FGA,TOV,PLUS_MINUS,AST,USG_PCT,Y
0,0.364654,0.331058,0.397395,0.694731,0.339809,0.415240,0.409157,0.645435,0.540045,0.525572,0
1,0.304896,0.376859,0.225032,0.437634,0.339809,0.351921,0.465928,0.202159,0.204132,0.329326,0
2,0.658228,0.648300,0.608492,0.605293,0.708391,0.726190,0.670808,0.726321,0.491915,0.727329,0
3,0.614272,0.610308,0.608492,0.731632,0.688276,0.644164,0.560714,0.845783,0.593746,0.490974,0
4,0.753611,0.657040,0.657989,0.869063,0.865016,0.860573,0.491722,0.477736,0.568187,0.811683,0
...,...,...,...,...,...,...,...,...,...,...,...
5329,0.411726,0.305702,0.225032,0.537071,0.443154,0.479934,0.581339,0.484965,0.593746,0.532402,0
5330,0.844027,0.773438,0.803580,0.736907,0.883919,0.878163,0.756013,0.672536,0.781773,0.842407,1
5331,0.449854,0.469857,0.423716,0.484695,0.443154,0.432405,0.267857,0.455946,0.204132,0.397378,0
5332,0.467574,0.248934,0.266438,0.281296,0.497437,0.531531,0.307468,0.583796,0.364866,0.359913,0


In [ ]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

#Hypertuning


In [ ]:
#Initiate the LR model with random hyperparameters
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
xgb0 = xgb.XGBClassifier(learning_rate = 0.05, n_estimators=300, max_depth=5)

In [ ]:
xgb0.fit(X,y)

XGBClassifier(learning_rate=0.05, max_depth=5, n_estimators=300)

In [ ]:
xgb0.score(X,y)

0.9981252343457068

In [ ]:
# You will need the following dependencies for applying Cross-validation and evaluating the cross-validated score

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Build the k-fold cross-validator

kfold = KFold(n_splits=3, random_state=2019,shuffle=True)

In [ ]:
result = cross_val_score(xgb0, X, y, cv=kfold, scoring='accuracy')
print('accuracy', result.mean())
result = cross_val_score(xgb0, X, y, cv=kfold, scoring='precision')
print('precision', result.mean())
result = cross_val_score(xgb0, X, y, cv=kfold, scoring='recall')
print('recall', result.mean())

accuracy 0.970753655793026
precision 0.7426964498984246
recall 0.6592864923747277


In [ ]:
learning_rate = [0.1,0.01,0.05]
n_estimators = [200,300,400]
max_depth = [3,5,10]
gamma =[5,6,7]
colsample_bytree=[1.0, 1.5]
param_grid = dict(learning_rate=learning_rate,n_estimators=n_estimators,max_depth=max_depth, gamma=gamma, colsample_bytree=colsample_bytree)

In [ ]:
import numpy as np
np.random.seed(7)
n_estimators1 = np.random.randint(200,300,400)
max_depth1 = np.random.randint(3,5,10)
gamma1 = np.random.randint(5,6,7)
param_grid1 = dict(n_estimators=n_estimators1,max_depth=max_depth1, gamma=gamma1)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
random = RandomizedSearchCV(estimator=xgb0, param_distributions=param_grid1, cv = 3, n_jobs=-1)

import time
start_time = time.time()
random_result = random.fit(X, y)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))
print("Execution time: " + str((time.time() - start_time)) + ' ms')

Best: 0.972066 using {'n_estimators': 277, 'max_depth': 4, 'gamma': 5}
Execution time: 14.184200286865234 ms


#Evaluation


In [ ]:
# balance the data
from imblearn.over_sampling import SMOTE

from sklearn import model_selection

In [ ]:
# resample/balance the data
sm = SMOTE(random_state = 2021) 
X_res, y_res = sm.fit_resample(X, y) 

In [ ]:
clf = xgb.XGBClassifier(learning_rate = 0.05,n_estimators= 277, max_depth= 4, gamma= 5)

In [ ]:
def my_eval(X, y, classifer = clf, k=10, scoring = 'f1'):
  '''
  return evaluation results (f1-score or ROC_AUC). 
  Built in k-fold evaluation.
  INPUTS:
  ----
  - X: features; DataFrame or Numpy ndarray;
  - y: target; DataFrame or Numpy ndarray;
  - classifier: any sklearn (or its add-on) based classifier
  - k: number of folds in cross validation
  - scoring: evaluation metric ('f1' default or 'roc_auc')
  OUTPUT:
  ----
  bias/variance score of selected metric.
  '''
  scores = []
  for i in range(50):
    #### generate random numbers to shuffle the data for training and test
    np.random.seed(2021)
    random_int = np.random.randint(0,3000)
    #### create cross validation folds
    kfold = model_selection.KFold(n_splits=k, random_state=random_int, shuffle=True)
    #### record the score
    score = model_selection.cross_val_score(clf, X=X, y=y, cv=kfold, scoring=scoring)
    scores.append(score)
  scores = np.array(scores)
  #### we need to calculate the bias (average score) and viariance (std)
  bias, variance = round(scores.mean(),4), round(scores.std(),4)
  return(bias, variance)

In [ ]:
# getting averaged f1_score from 10-fold CV (default)
my_eval(X_res, y_res, clf, 10)

(0.9677, 0.0026)

In [ ]:
# getting averaged ROC_AUC from 10-fold CV
my_eval(X_res, y_res, clf, 10, 'roc_auc')

(0.9921, 0.0018)

In [ ]:
clf = xgb.XGBClassifier(n_estimators= 277, max_depth= 4, gamma= 5)

In [ ]:
# getting averaged f1_score from 10-fold CV (default)
my_eval(X_res, y_res, clf, 10)

(0.9676, 0.0021)

In [ ]:
# getting averaged ROC_AUC from 10-fold CV
my_eval(X_res, y_res, clf, 10, 'roc_auc')

(0.9923, 0.0018)